In [2]:
import torch
import getTransUnet
import loadData as dl
import train
from torchinfo import summary
import os
import time
import numpy as np
import prediction
from evaluation import save_eval_result
from unet import UNet
from pyramidDilate import PyramidDilate
from dilated_net import DilatedNet
from dilated_net_6 import DilatedNet as DilatedNet6
from dilated_net_8 import DilatedNet as DilatedNet8
from dilated_net_12 import DilatedNet as DilatedNet12
from unet import UNet

KCONST = 3
NEGATIVENUM = 39
POSITIVENUM = 6
dateTime = time.strftime("%Y%m%d_%H_%M", time.localtime())
resultDir = os.path.join('../result/TransUnet', dateTime)

negSliceList = []
posSliceList = []
def get_set_slice(KConst, NSize, PSize, Round):
    if negSliceList != []:
        return negSliceList[Round], posSliceList[Round]
    negID, posID = 0, 0
    remainder = [NSize%KConst, PSize%KConst]
    for i in range(KConst):
        foldSize = [NSize//KConst, PSize//KConst]
        if (i < remainder[0]):
            foldSize[0]+=1
        if (i < remainder[1]):
            foldSize[1]+=1
        oldNegID, oldPosID = negID, posID
        negID, posID = negID+foldSize[0], posID+foldSize[1]
        negSliceList.append([oldNegID, negID])
        # Positive cases are placed after negative cases
        # The actual index needs to be added with the previous number of negative cases
        posSliceList.append([oldPosID+NSize, posID+NSize]) 
    return negSliceList[Round], posSliceList[Round]

def train_by_model(model, model_name, image_set, round, base_lr=0.01):
    test_slice = get_set_slice(3, NEGATIVENUM, POSITIVENUM, round)
    print(test_slice)
    train_dataset = dl.SegmentationDataset(image_set, test_slice, flag='train')
    test_dataset = dl.SegmentationDataset(image_set, test_slice, flag='test')
    model = train.train(model, device, train_dataset, num_classes=11, base_lr=base_lr, epochs=200)
    torch.save(model.state_dict(), os.path.join(resultDir,model_name+'.pth'))
    _, _, label_img = image_set.getImageSet(test_slice, 'test')
    fileName_set = image_set.getFileNameSet(test_slice, 'train')
    print("file name train", fileName_set)
    fileName_set = image_set.getFileNameSet(test_slice, 'test')
    print("file name test", fileName_set)

    #print("-------",np.shape(image_set.fileNames), np.shape(fileName_set))
    test_raw_img = image_set.getRawImageSet(test_slice, 'test')
    pred_img, mask_img = prediction.multi_class_predict(model, test_dataset, device, 11)
    subResDir = os.path.join(resultDir, 'iteration '+str(round), model_name)
    meanPrec, meanReca, meanFmea = save_eval_result(predict_image=pred_img,
                                                    gt_image=label_img,
                                                    test_filenames=fileName_set,
                                                    resDir=subResDir,
                                                    mask_image=mask_img,
                                                    overlay_on=True,
                                                    ori_image=test_raw_img)
    
os.makedirs(resultDir, exist_ok=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_set = dl.ImageSet(dataset_dir='../dataset')
for i in range(KCONST):
    # def train_md_convnet(block_num, dilation_rates, layer_num, net_name):
    #     start_time = time.time()
    #     model = DilatedNet(1, 1, block_num = block_num, dilation_rates = dilation_rates, layer_num = layer_num)
    #     model = model.to(device=device)
    #     train_by_model(model, net_name, image_set, i, base_lr=0.01)
    #     end_time = time.time()
    #     with open(os.path.join(resultDir, net_name+'.txt'), "a+") as f:
    #         f.writelines("fold" + str(i) + ": " + str(end_time-start_time)+'\n')

    '''
    train_md_convnet(10, [1,3], 2, "md_block10_layer1+3")
    train_md_convnet(10, [1,2,3,5,7], 5, "md_block10_layer1+2+3+5+7")
    train_md_convnet(10, [1,5], 2, "md_block10_layer1+5")
    train_md_convnet(10, [1,2], 2, "md_block10_layer1+2")
    train_md_convnet(10, [1,2,5], 3, "md_block10_layer1+2+5")
    train_md_convnet(8, [1,2,5,9], 4, "md_block8_layer1+2+5+9")
    train_md_convnet(6, [1,2,5,9], 4, "md_block6_layer1+2+5+9")
    '''
    # for j in range(1):
    #     train_md_convnet(10, [1,2,5,9], 4, "md_block10_layer1+2+5+9_"+str(j))
    '''
    train_md_convnet(12, [1,2,5,9], 4, "md_block12_layer1+2+5+9")
    train_md_convnet(10, [1,2,3,5], 4, "md_block10_layer1+2+3+5")
    train_md_convnet(10, [1,2,3,5,7,11], 6, "md_block10_layer1+2+3+5+7+11")
    '''
    start_time = time.time()
    model = getTransUnet.get_transNet(11, bMask=False)
    model = model.to(device=device)
    model_name = 'transunet'
    train_by_model(model, model_name, image_set, i, base_lr=0.01)
    end_time = time.time()
    with open(os.path.join(resultDir, model_name+'.txt'), "a+") as f:
        f.writelines("fold" + str(i) + ": " + str(end_time-start_time)+'\n')
    # start_time = time.time()
    # model = UNet(1, 1)
    # model = model.to(device=device)
    # model_name = 'unet'
    # train_by_model(model, model_name, image_set, i, base_lr=0.01)
    # end_time = time.time()
    # with open(os.path.join(resultDir, model_name+'.txt'), "a+") as f:
    #     f.writelines("fold" + str(i) + ": " + str(end_time-start_time)+'\n')
    '''
    model = DilatedNet8(1, 1)
    model = model.to(device=device)
    train_by_model(model, 'multiscaledilated8', image_set, i, base_lr=0.01)
    model = DilatedNet(1, 1)
    model = model.to(device=device)
    train_by_model(model, 'multiscaledilated10', image_set, i, base_lr=0.01)
    model = DilatedNet(1, 1)
    model = model.to(device=device)
    train_by_model(model, 'multiscaledilated10_1', image_set, i, base_lr=0.01)
    model = DilatedNet(1, 1)
    model = model.to(device=device)
    train_by_model(model, 'multiscaledilated10_2', image_set, i, base_lr=0.01)
    model = DilatedNet(1, 1)
    model = model.to(device=device)
    train_by_model(model, 'multiscaledilated10_3', image_set, i, base_lr=0.01)
    model = DilatedNet(1, 1)
    model = model.to(device=device)
    train_by_model(model, 'multiscaledilated10_4', image_set, i, base_lr=0.01)
    
    '''
    #model = DilatedNet12(1, 1)
    #model = model.to(device=device)
    #train_by_model(model, 'multiscaledilated12', image_set, i, base_lr=0.01)


reading images...
bMask False
([0, 13], [39, 41])
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[39, 40]
=========getImageSet========= ([0, 13], [39, 41]) 26 4 30
=========== train (30, 256, 256, 1) ===========
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[39, 40]
=========== test (15, 256, 256, 1) ===========


Epoch 1/200: 100%|██████████| 21/21 [00:01<00:00, 13.50img/s]


 epoch: 1, train_loss: 0.6852



 epoch: 1, val_loss: 3.7572


Epoch 2/200: 100%|██████████| 21/21 [00:01<00:00, 13.79img/s]

...

Epoch 200/200: 100%|██████████| 21/21 [00:01<00:00, 13.22img/s]


 epoch: 200, train_loss: 0.0184



 epoch: 200, val_loss: 0.1906
[26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]
[43, 44]
=========== test (15, 256, 256, 1) ===========
file name train ['N2-9.png', 'N2-8.png', 'N3-10.png', 'N3-8.png', 'N3-3.png', 'N1-6.png', 'N2-2.png', 'N1-3.png', 'N4-5.png', 'N2-3.png', 'N4-3.png', 'N4-8.png', 'N1-1.png', 'N4-2.png', 'N3-7.png', 'N2-6.png', 'N2-1.png', 'N3-2.png', 'N4-1.png', 'N1-4.png', 'N3-6.png', 'N3-5.png', 'N4-6.png', 'N2-4.png', 'N2-10.png', 'N2-7.png', 'N5-6.png', 'N5-2.png', 'N5-1.png', 'N5-3.png']
file name test ['N1-8.png', 'N1-7.png', 'N4-7.png', 'N3-9.png', 'N1-10.png', 'N4-9.png', 'N3-1.png', 'N3-4.png', 'N2-5.png', 'N1-5.png', 'N1-9.png', 'N1-2.png', 'N4-4.png', 'N5-5.png', 'N5-4.png']


15it [00:00, 15.40it/s]


Mean class 1 mean_dice 8.496616 mean_hd95 0.000000
Mean class 2 mean_dice 0.000000 mean_hd95 0.000000
Mean class 3 mean_dice 9.475165 mean_hd95 0.000000
Mean class 4 mean_dice 1.505913 mean_hd95 0.000000
Mean class 5 mean_dice 6.949150 mean_hd95 0.000000
Mean class 6 mean_dice 9.936995 mean_hd95 0.000000
Mean class 7 mean_dice 1.919259 mean_hd95 0.000000
Mean class 8 mean_dice 4.141389 mean_hd95 0.000000
Mean class 9 mean_dice 0.056563 mean_hd95 0.000000
Mean class 10 mean_dice 1.419465 mean_hd95 0.000000
